# Generative Adversarial Networks (GANs)

Here we demonstrate the basic construction, training, and evaluation of a GAN using Pytorch.

In [1]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

Random Seed:  999


In [2]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3 (nc)
num_channels = 3 

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator (ngf)
num_gen_feature_maps = 64

# Size of feature maps in discriminator (ndf)
num_dis_feature_maps = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02) # why are batchnorms initialized to have a mean of 1 vs convolutional layers are set to have a mean of 1
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
    
    def __init__(self, ngpus):
        super(Generator,self).__init__()
        self.ngpus = ngpus
        
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, num_gen_feature_maps*8, 4, 1, 0, bias=False)
            nn.BatchNorm2d(num_gen_feature_maps*8),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(num_gen_feature_maps*8, num_gen_feature_maps*4, 4, 2, 0, bias=False)
            nn.BatchNorm2d(num_gen_feature_maps*4),
            nn.ReLU(True),
            
            
            nn.ConvTranspose2d(num_gen_feature_maps*4, num_gen_feature_maps*4, 2, 4, 0, bias=False)
            nn.BatchNorm2d(num_gen_feature_maps*2),
            nn.ReLU(True),
            
        )
        